In [ ]:
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import os
import cv2
from tqdm import tqdm

# use seaborn plotting defaults
import seaborn as sns; sns.set()

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.decomposition import KernelPCA, PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog

In [ ]:
#директории train и test датасетов
dirname_train = "/gdrive/My Drive/StartML_AMI/train"
dirname_test = "/gdrive/My Drive/StartML_AMI/test"

Тренировочный датасет довольно маленький по сравнению с тестовым, поэтому мы используем агументацию изображений (варьируем поворот и смещение, яркость и контраст, отражение по горизонтали). В тестовом датасете также присутствуют зашумленные фотографии, поэтому добавляем Гауссовский шум, его интенсивность подбиралась опытынм путем (параметр var_limit). 

In [ ]:
import random
import albumentations as A

from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)

def get_aug(image):
    angle = np.arange(-10,11,1)
    angle0 = random.choice(angle)

    shift = 0.01*np.arange(-10,11,1)
    shift0 = random.choice(shift)

    transform = A.Compose([
    A.ShiftScaleRotate(shift_limit=[shift0,shift0], scale_limit=0, 
                        rotate_limit=[angle0,angle0], p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.3),
    A.GaussNoise(var_limit=(10,250), p=0.5)
    ])
    return transform(image=image)['image']

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


Upload and applying Haar cascade with augmentations.

In [ ]:
X_train, y_train, X_train3 = [], [], []

X_train_Aug, y_train_Aug = [], []
no_face = 0
filelist = os.listdir(dirname_train)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')

for fname in filelist:
    img = cv2.imread(os.path.join(dirname_train, fname), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
    X_train.append(img)
    y_train.append(int(fname.split('label')[1].split('.')[0]))

for (a,b) in zip(X_train,y_train):
    for i in range(4):
        X_train3.append(get_aug(a))
        y_train_Aug.append(b)

for img in X_train3:    
    face = face_cascade.detectMultiScale(img)
    
    for (a, b, w, h) in face:
        cv2.rectangle(img, (a, b), (a+w, b+h), (0, 0, 255), 2)
        face = img[b:b + h, a:a + w]

    try:
        h, w = face.shape
        size = min(h, w)
        h0 = int((h - size) / 2)
        w0 = int((w - size) / 2)
            
        img = face[h0: h0 + size, w0: w0 + size]        

    except:
        no_face+=1

    img = cv2.resize(img, (64, 64), interpolation = cv2.INTER_AREA)    
    X_train_Aug.append(img)
    

print(no_face,'faces were not found')

len(X_train_Aug), len(y_train_Aug)

FileNotFoundError: ignored

In [ ]:
bins = np.arange(0, 21, 1) 

plt.hist(y_train, bins=bins, alpha=1)
plt.xlabel('Classes')
plt.ylabel('Counts')

plt.show()

In [ ]:
fig, axx = plt.subplots(1, 8, figsize=(20, 10))

for i in range(8):
    img = X_train_Aug[i+62]
    axx[i].imshow(img)

Downscale test dataset to (64, 64)

In [ ]:
def sort_by_index(fname):
    return int(fname.split('.')[0].split('img')[1])

In [ ]:
X_test = []
Names = []
no_face = 0

for fname in tqdm(sorted(os.listdir(dirname_test), key=sort_by_index)):
    img = cv2.imread(os.path.join(dirname_test, fname), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA) 
    Names.append(fname)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')
    face = face_cascade.detectMultiScale(img)
    
    for (a, b, w, h) in face:
        cv2.rectangle(img, (a, b), (a+w, b+h), (0, 0, 255), 2)
        face = img[b:b + h, a:a + w]

    try:
        h, w = face.shape
        size = min(h, w)
        h0 = int((h - size) / 2)
        w0 = int((w - size) / 2)
            
        img = face[h0: h0 + size, w0: w0 + size]
              
    except:
        no_face+=1
    
    img = cv2.resize(img, (64, 64), interpolation = cv2.INTER_AREA)
    X_test.append(img)
    
print(no_face,'faces were not found')
len(X_test)

In [ ]:
fig, axx = plt.subplots(1, 4, figsize=(10, 5))

for i in range(4):
    img = X_test[i+90]
    axx[i].imshow(img)

Searching attributes via HOG

In [ ]:
hog_img_train = []
for i in range(len(X_train_Aug)):
    fd, hog_image = hog(X_train_Aug[i], orientations=8, pixels_per_cell=(8, 8),
                	cells_per_block=(2, 2), visualize=True)
    hog_img_train.append(hog_image)

In [ ]:
hog_img_test = []
for i in range(len(X_test)):
    fd, hog_image = hog(X_test[i], orientations=8, pixels_per_cell=(8, 8),
                	cells_per_block=(2, 2), visualize=True)
    hog_img_test.append(hog_image)

In [ ]:
fig, axx = plt.subplots(1, 8, figsize=(20, 10))

for i in range(8):
    img = hog_img_test[i+45]
    axx[i].imshow(img)

Model fitting

In [ ]:
X_train_flatten = np.array([el.ravel() for el in hog_img_train])
X_test_flatten = np.array([el.ravel() for el in hog_img_test])

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train_flatten, y_train_Aug, stratify=y_train_Aug, random_state=42)

Appying PCA

In [ ]:
pca = PCA().fit(X_tr, y_tr)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.axhline(y=0.95, color='r', linestyle='-')
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')

In [ ]:
scl = StandardScaler()
pca = KernelPCA()
model_svm = SVC(probability=True)

pipe = Pipeline(steps=[('scl', scl), ('pca', pca), ('svm', model_svm)])

param_grid = {
    "pca__n_components": [148],
    "pca__kernel": ['rbf'],
    'svm__C': [21],
    'svm__kernel': ['rbf'],
}

grid = GridSearchCV(estimator = pipe, param_grid = param_grid, 
                           scoring = 'accuracy', cv = 10, 
                           n_jobs = -1, verbose = 2)

grid.fit(X_tr, y_tr)

print(f"best mean cross-validation score: {grid.best_score_}")
print(f"best parameters: {grid.best_params_}")
print(f"val-set score: {grid.score(X_val, y_val):.3f}")

In [ ]:
from catboost import CatBoostClassifier

cbc = CatBoostClassifier(task_type='GPU')

parameters = {
                 'learning_rate' : [0.18,0.19,0.20, 0.21],
                  'depth': [4, 6,]
                  #'l2_leaf_reg': [1, 3, 5, 7, 9],
                  #'border_count' : [128, 142, 168, 186]
              
                 }

grid_cbc = cbc.grid_search(parameters, X_tr, y_tr, plot=True)
#grid_cbc.fit(X_tr, y_tr)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Streaming output truncated to the last 5000 lines.
15:	learn: 1.7210710	test: 2.0200500	best: 2.0200500 (15)	total: 703ms	remaining: 43.3s
16:	learn: 1.6753269	test: 1.9862384	best: 1.9862384 (16)	total: 744ms	remaining: 43s
17:	learn: 1.6260733	test: 1.9495444	best: 1.9495444 (17)	total: 788ms	remaining: 43s
18:	learn: 1.5891085	test: 1.9289209	best: 1.9289209 (18)	total: 829ms	remaining: 42.8s
19:	learn: 1.5519953	test: 1.9026267	best: 1.9026267 (19)	total: 870ms	remaining: 42.6s
20:	learn: 1.5152572	test: 1.8775192	best: 1.8775192 (20)	total: 916ms	remaining: 42.7s
21:	learn: 1.4865282	test: 1.8565814	best: 1.8565814 (21)	total: 960ms	remaining: 42.7s
22:	learn: 1.4588552	test: 1.8362540	best: 1.8362540 (22)	total: 1000ms	remaining: 42.5s
23:	learn: 1.4301689	test: 1.8190480	best: 1.8190480 (23)	total: 1.04s	remaining: 42.3s
24:	learn: 1.3900153	test: 1.7876498	best: 1.7876498 (24)	total: 1.08s	remaining: 42.3s
25:	learn: 1.3760127	test: 1.7823887	best: 1.7823887 (25)	total: 1.14s	r

In [ ]:
grid_cbc.get('params')

{'learning_rate': 0.2}

Saving prediction to csv

In [ ]:
pred = cbc.predict(X_test_flatten)
pred_df = pd.DataFrame(list(zip(Names, pred)), columns = ['img', 'label'])

pred_df.to_csv("sec_submission.csv", index=False)

In [ ]:

pred_df.to_csv("first_submission.csv", index=False)